In [ ]:
import shutil
import os
import numpy as np
from datetime import datetime
import matplotlib.pylab as plt
import tensorflow as tf

import sys
sys.path.append(os.path.abspath("../../framework"))
sys.path.append(os.path.abspath("../../concrete"))

In [ ]:
from AsmAction import AsmAction
from AsmSimulator import AsmSimulator
from ConcObservation import ConcObservation
from ConcAgentFactory import ConcAgentFactory
from ConcBuildOrder import ConcBuildOrder
from ConcEnvrionmentFactory import ConcEnvironmentFactory
from ConcRewardGiverFactory import ConcRewardGiverFactory
from ConcTrainerFactory import ConcTrainerFactory
from ConcValueFunctionApproximatorFactory import ConcValueFunctionApproximatorFactory
from framework import Trainer, ObservationSequence

## S100: Check the responce of Asm Simulator to the actions

In [ ]:
asmSimulator =  AsmSimulator(amplitudePeriodicDv=2.0)

In [ ]:
nDays = 40 # [day]
gain = 2.0 # 
bias = -3.2

In [ ]:
def testSimulation(asmSimulator, max_step, nDays):
    asmSimulator.init()
    asmSimulator.odeHandler.set_integrator('dopri5', method="adams", max_step=max_step)

    Do = []
    S_O2 = []
    S_NH4 = []
    X = []
    for _ in range(96*nDays): # 96[step] = [1day]

        observation = asmSimulator.observe()
        e = observation.getValue()
        u = gain * e + bias
        action = AsmAction(np.ones((1,1))*u)

        asmSimulator.update(action)        
        observation = asmSimulator.observe()
        S_NH4.append(float(observation.S_NH4))
        Do.append(float(action.getActionOnEnvironment()))
        S_O2.append(asmSimulator.x[7])
        X.append(asmSimulator.x)

    S_NH4 = np.stack(S_NH4, axis=0) # (*, )
    Do = np.stack(Do, axis=0) # (*, )
    S_O2 = np.stack(S_O2, axis=0) # (*,)
    X = np.stack(X, axis=0) # (*, 17)
    
    return S_NH4, Do

In [ ]:
for max_step in [1e-4, 1e-7]:
    S_NH4, Do = testSimulation(asmSimulator, max_step, nDays)

    t = np.linspace(0, nDays, 96*nDays)

    fig = plt.figure()
    fig.clf()
    ax1 = fig.add_subplot(2,1,1)
    ax2 = fig.add_subplot(2,1,2)
    ax1.plot(t, S_NH4)
    ax2.plot(t, Do)

    ax1.set_ylabel('NH4')
    ax2.set_ylabel('DO')
    plt.tight_layout()

    plt.savefig('simulation_max_step=%1.0e.png' % max_step)

    fig = plt.figure()
    fig.clf()
    ax1 = fig.add_subplot(2,1,1)
    ax2 = fig.add_subplot(2,1,2)
    ax1.plot(t[-96*2:], S_NH4[-96*2:])
    ax2.plot(t[-96*2:], Do[-96*2:])

    ax1.set_ylabel('NH4')
    ax2.set_ylabel('DO')
    plt.tight_layout()

    plt.savefig('simulation_max_step=%1.0e_lastTwoDays.png' % max_step)